## Soccer Odds Prediction Fast API

How to use: Execute all notebook cells and click on the "....ngrok.io" webpage link (last cell) to access the API. Add a "/docs" to the link to try the API out.

API Input: 
- Match timestamp (please use UTC-times!)
- Home Team Name
- Away Team Name

The API looks for the corresponding match data (Features) out of given data and based on this, it predicts the match outcome probabilities and recommended odds.

API Output: 
- Prediction Result (0/1/2) = (away win / draw / home win)
- Prediction Probabilities
- Suggested Odds

### Installations & Imports

we need to install colabcode and fastapi

In [8]:
# install colabcode (can take some minutes)
!pip install colabcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 8.0 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 8.3 MB 30.6 MB/s 
     |████████████████████████████████| 423 kB 72.7 MB/s 
     |████████████████████████████████| 344 kB 75.2 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 9.7 MB 45.5 MB/s 
     |████████████████████████████████| 58 kB 7.1 MB/s 
     |████████████████████████████████| 132 kB 31.3 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 561 kB 72.6 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 131 kB 68.7 MB/s 
     |████████████████████████████████| 131 kB 68.0 MB/s 
     |████████████████████████████████| 130 kB 67.6 MB/s 
     |████████████████████████████████| 13

In [9]:
# install fastapi
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [2]:
# module imports
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
from colabcode import ColabCode
from pydantic import BaseModel
from fastapi import FastAPI
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

### Data Preperation

In [3]:
# download match data (created in other notebook)
!gdown 1QdDoV4JPg_YCDbJ6eQrbpvFkGeXT39cd
# download ML-model (trained in other notebook)
!gdown 1ZVwIZogxYlpS3nEqcde2o6IWkQcXvkFk
# load data to dataframe
test_soccer_data = pd.read_csv("/content/test_soccer_data.csv").drop(columns=["Unnamed: 0"])

Downloading...
From: https://drive.google.com/uc?id=1QdDoV4JPg_YCDbJ6eQrbpvFkGeXT39cd
To: /content/test_soccer_data.csv
100% 32.7M/32.7M [00:00<00:00, 57.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZVwIZogxYlpS3nEqcde2o6IWkQcXvkFk
To: /content/soccer_xgb_model.pkl
100% 386k/386k [00:00<00:00, 149MB/s]


In [12]:
# Data Preperation

target = test_soccer_data['target']
  
y_pipeline_data = np.array(target)

x_pipeline_data = test_soccer_data.drop(columns=['id', 'target', 'home_team_name', 'away_team_name', 'league_name', 'match_date', 'home_team_coach_id', 'away_team_coach_id'])

In [13]:
# Data Pipeline for the Model Prediction
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler())
])

pipeline.fit(x_pipeline_data)
x_data = pipeline.transform(x_pipeline_data)

### API Code

In [14]:
# define API input schema
class Soccer(BaseModel):
    match_date: str
    home_team: str 
    away_team: str 
    class Config:
        schema_extra = {
            "example": {
                "match_date": "2021-04-24 13:30:00", 
                "home_team": "Mainz 05", 
                "away_team": "Bayern München"
            }
        }

In [15]:
# create Fast API
app = FastAPI()

def get_match_data(match_date, home_team, away_team):
    # get corresponding data to match input
    row_number = test_soccer_data.loc[(test_soccer_data["match_date"] == match_date) & (test_soccer_data["home_team_name"] == home_team) & (test_soccer_data["away_team_name"] == away_team)].index.tolist()[0]
    return row_number

@app.on_event("startup")
def load_model():
    # load trained soccer prediction model
    global model
    model = pickle.load(open("/content/soccer_xgb_model.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'Welcome to our soccer odds prediction API. Important note: 2 = win home-team; 1 = draw, 0 = win away-team'}


@app.post('/predict')
def get_match_prediction(data: Soccer):
    # create prediction

    received = data.dict()
    match_date = received["match_date"]
    home_team = received["home_team"]
    away_team = received["away_team"]

    row_number = get_match_data(match_date,home_team,away_team)

    pred_data = [x_data[row_number]]

    # prediction result (0=away win, 1=draw, 2=home win)
    pred_result = str(model.predict(pred_data)[0])
    # prediction probabilities
    pred_prob_away = str(np.round((model.predict_proba(pred_data)[0][0]),2))
    pred_prob_draw = str(np.round((model.predict_proba(pred_data)[0][1]),2))
    pred_prob_home = str(np.round((model.predict_proba(pred_data)[0][2]),2))
    # calculated odds
    suggested_odd_away = str(np.round((1/(model.predict_proba(pred_data)[0][0])*0.9),2))
    suggested_odd_draw = str(np.round((1/(model.predict_proba(pred_data)[0][1])*0.9),2))
    suggested_odd_home = str(np.round((1/(model.predict_proba(pred_data)[0][2])*0.9),2))
      
    return {'pred_result': pred_result, 'pred_prob_win_away_team': pred_prob_away, 
            'pred_prob_draw': pred_prob_draw, 'pred_prob_win_home_team': pred_prob_home,
            'suggested_odd_away': suggested_odd_away, 'suggested_odd_draw': suggested_odd_draw, 'suggested_odd_home': suggested_odd_home}
    

In [16]:
# server
server = ColabCode(port=10000, code=False)

In [17]:
# to use ngrok in colab, authtoken must be saved in config file
!ngrok authtoken 2BCqLq4oBfPnWubMP9D3D5qLkOF_4TYAeuu1gk4CantDTLjXb

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
# some matches to try the API out:
(test_soccer_data[test_soccer_data["league_name"] == "Bundesliga"]).tail(5)

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,...,away_draw_5,home_loser_5,away_loser_5,home_wins_last_5,away_wins_last_5,home_draws_last_5,away_draws_last_5,home_losses_last_5,away_losses_last_5,league_name_encoded
23794,16840342,0,Wolfsburg,Borussia Dortmund,2021-04-24 13:30:00,Bundesliga,82,0,51518.0,417317.0,...,0.0,0.0,1.0,3.0,3.0,0.0,0.0,2.0,2.0,87
23796,16840346,2,Mainz 05,Bayern München,2021-04-24 13:30:00,Bundesliga,82,0,29535.0,37352367.0,...,0.0,0.0,1.0,3.0,3.0,2.0,1.0,0.0,1.0,87
24056,16840341,2,Bayer Leverkusen,Eintracht Frankfurt,2021-04-24 16:30:00,Bundesliga,82,0,459565.0,455978.0,...,0.0,1.0,0.0,2.0,4.0,1.0,0.0,2.0,1.0,87
24555,16840339,2,RB Leipzig,Stuttgart,2021-04-25 13:30:00,Bundesliga,82,0,458813.0,45305.0,...,0.0,0.0,1.0,2.0,1.0,1.0,0.0,2.0,4.0,87
24708,16840340,2,Borussia M'gladbach,Arminia Bielefeld,2021-04-25 16:00:00,Bundesliga,82,0,29710.0,37336957.0,...,0.0,0.0,1.0,3.0,2.0,1.0,2.0,1.0,1.0,87


In [ ]:
# run the Fast API
# open the ".ngrok.io" URL to access the API
# to read the docu and try the API out, add a "/docs" to the link
server.run_app(app=app)

INFO:     Started server process [70]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://bda9-34-82-83-41.ngrok.io" -> "http://localhost:10000"
INFO:     84.119.216.155:0 - "GET / HTTP/1.1" 200 OK
INFO:     84.119.216.155:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     84.119.216.155:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     84.119.216.155:0 - "POST /predict HTTP/1.1" 200 OK


Public URL: NgrokTunnel: "https://4a61-34-86-60-158.ngrok.io" -> "http://localhost:10000"
